# Transfer learning:
- Machine learning method where a model developed for a task is reused as the starting point for a model on a second task.

# || Loading Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, time, random, cv2
from pathlib import Path
from PIL import Image
import imgaug as ia
from imgaug import augmenters as iaa
from tqdm import tqdm

from keras.models import Model
from keras.layers import (Conv2D, Input, Dense, Flatten, Dropout, GlobalAveragePooling2D,
                          Activation, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU)
from keras.optimizers import Adam, SGD, RMSprop
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


# || Configuration

In [2]:
%matplotlib inline
pd.set_option('max_colwidth', 400)
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 16
t_start = time.time()

DATA_DIR = "../input/rice-diseases-image-dataset/LabelledRice/Labelled/"
CATEGORIES = os.listdir(DATA_DIR)
Ncategories = len(CATEGORIES)

In [3]:
# ----------------#
# Hyperparameters #
# ----------------#
Target_shape = (224, 224)
Test_size = 0.05
Nepochs = 10
Batch_size = 32
Learning_rate = 1e-04

# || Data Preparation

In [4]:
data = []
for category_id, category in enumerate(CATEGORIES):
    for file in os.listdir(os.path.join(DATA_DIR, category)):
        if file == ".directory": continue
        data.append(['{}{}/{}'.format(DATA_DIR, category, file), category_id, category])
data = pd.DataFrame(data, columns=['img_path', 'category_id', 'category'])
data['category_cat'] = list(to_categorical(data["category_id"], Ncategories))

data.sample(n=10, random_state=100).head(n=5)

,img_path,category_id,category,category_cat
2187,../input/rice-diseases-image-dataset/LabelledRice/Labelled/Healthy/IMG_20190419_135833.jpg,2,Healthy,"[0.0, 0.0, 1.0, 0.0]"
2709,../input/rice-diseases-image-dataset/LabelledRice/Labelled/Healthy/IMG_20190420_200441.jpg,2,Healthy,"[0.0, 0.0, 1.0, 0.0]"
1830,../input/rice-diseases-image-dataset/LabelledRice/Labelled/Healthy/IMG_20190419_172317.jpg,2,Healthy,"[0.0, 0.0, 1.0, 0.0]"
452,../input/rice-diseases-image-dataset/LabelledRice/Labelled/LeafBlast/IMG_20190419_105827.jpg,0,LeafBlast,"[1.0, 0.0, 0.0, 0.0]"
2973,../input/rice-diseases-image-dataset/LabelledRice/Labelled/BrownSpot/IMG_20190420_190255.jpg,3,BrownSpot,"[0.0, 0.0, 0.0, 1.0]"


# || Dividing the Data To Train, Val and Test

In [5]:
train, val = train_test_split(data, stratify=data.category, test_size=Test_size)
train, test = train_test_split(train, stratify=train.category, test_size=Test_size)
print("Training images: {}\nValidating images: {}\nTesting images: {}".format(len(train), len(val), len(test)))

Training images: 3027
Validating images: 168
Testing images: 160


# || Data Augmentation

In [6]:
# https://imgaug.readthedocs.io/en/latest/source/examples_keypoints.html
# https://github.com/aleju/imgaug
# https://imgaug.readthedocs.io/en/latest/source/augmenters.html
seq = iaa.Sequential([
    iaa.OneOf([
        iaa.Fliplr(0.5), # horizontally flip
        iaa.Flipud(0.5),
        iaa.Affine(rotate=45)
    ]),
    iaa.SomeOf(2, [
        iaa.AdditiveGaussianNoise(scale=0.2*255),
        iaa.Add(50, per_channel=True),
        iaa.Sharpen(alpha=0.2),
        iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)),
        iaa.Noop(),
        iaa.Dropout(p=(0, 0.2), per_channel=0.5),
        iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5),
    ])
    # More as you want ...
])
# seq_det = seq.to_deterministic()

# || Data Generator

In [7]:
def preprocess_input(img):
    #--- Rescale Image --- Rotate Image --- Resize Image --- Flip Image --- PCA etc.
    image = cv2.resize(img, Target_shape, interpolation = cv2.INTER_AREA)
    image = image/255.
    return(image)

def get_bach_balenced_idx(df, batch_size, nCategories):
    k = batch_size // nCategories
    idx = []
    for i in range(nCategories):
        _ = df[df['category_id']==i].sample(n=k).index.to_list()
        idx.extend(_)
    return idx
    
def batch_generator(df, batch_size = 16):
    while True:
        # Select files (paths/indices) for the batch
        df.reset_index(drop=True, inplace=True)
        idx = get_bach_balenced_idx(df, batch_size, 4)
        
        batch_paths = df.iloc[list(idx), :]["img_path"].values
        batch_outputs = df.iloc[list(idx), :]["category_cat"].values
        
        batch_input = []
        batch_output = [] 

        # Read in each input, perform preprocessing and get labels
        for input_path, output_ in zip(batch_paths, batch_outputs):
            img = cv2.imread(str(input_path))
            input_ = preprocess_input(img)
            batch_input += [ input_ ]
            batch_output += [ output_ ]
        # Return a tuple of (input,output) to feed the network
        batch_x = np.array( batch_input )
        # Augmentation for batch of images
        batch_x = seq.augment_images(batch_x)
        batch_y = np.array( batch_output )
        
        yield( batch_x, batch_y )

In [8]:
# https://keras.io/applications/
from keras.applications import (Xception, VGG16, VGG19, ResNet50, InceptionV3, 
                                InceptionResNetV2, MobileNet, DenseNet121, NASNetMobile)
def build_model(ModelName, InputShape = Target_shape):
    inp = Input((InputShape[0],InputShape[1],3))
    if ModelName == "Xception":
        base_model = Xception(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "VGG16":
        base_model = VGG16(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "VGG19":
        base_model = VGG19(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "ResNet50":
        base_model = ResNet50(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "InceptionV3":
        base_model = InceptionV3(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "InceptionResNetV2":
        base_model = InceptionResNetV2(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "MobileNet":
        base_model = MobileNet(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "DenseNet121":
        base_model = DenseNet121(input_tensor = inp, include_top=False, weights='imagenet')
    elif ModelName == "NASNetMobile":
        base_model = NASNetMobile(input_tensor = inp, include_top=False, weights='imagenet')
        
    # frozen the first .8% layers
    NtrainableLayers = round(len(base_model.layers)*0.8)
    for layer in base_model.layers[:NtrainableLayers]:
        layer.trainable = False
    for layer in base_model.layers[NtrainableLayers:]:
        layer.trainable = True
    
    x_model = base_model.output
    x_model = GlobalAveragePooling2D(name='globalaveragepooling2d')(x_model)
    # x_model = Dense(1024, activation='relu',name='fc1_Dense')(x_model)
    # x_model = Dropout(0.5, name='dropout_1')(x_model)
    # x_model = Dense(256, activation='relu',name='fc2_Dense')(x_model)
    x_model = Dropout(0.1, name='dropout_2')(x_model)

    predictions = Dense(Ncategories, activation='softmax',name='output_layer')(x_model)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

# || Training

In [ ]:
# Callbacks for training: Make sure the training works well and doesnt run too long or overfit too much
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

MName = ("Xception")
BestModelWeightsPath = "{}_weights.best.hdf5".format(MName)

checkpoint = ModelCheckpoint(
    BestModelWeightsPath, 
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    mode='max', 
    save_weights_only = True
)
reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_accuracy', 
    factor=0.2, 
    patience=3, 
    verbose=1, 
    mode='max', 
    cooldown=2, 
    min_lr=1e-7
)
early = EarlyStopping(
    monitor="val_accuracy", 
    mode="max", 
    patience=4
)

print(f"\n Transfer Learning with: {MName}: \n")

model = build_model(MName, InputShape = Target_shape)

model.compile(optimizer=Adam(lr=Learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

train_gen = batch_generator(train, Batch_size)
val_gen = batch_generator(val, Batch_size)

History = model.fit_generator(
    generator= train_gen,
    steps_per_epoch=train.shape[0]//Batch_size,
    validation_data=val_gen,
    validation_steps=val.shape[0]//Batch_size,
    epochs = Nepochs,
    callbacks = [checkpoint, early, reduceLROnPlat]
)

print("\nFine tuning")
for l in model.layers[:]:
    l.trainable = True
    
model.compile(optimizer=SGD(lr=Learning_rate*1e-02, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

History = model.fit_generator(
    generator= train_gen,
    steps_per_epoch=train.shape[0]//Batch_size,
    validation_data=val_gen,
    validation_steps=val.shape[0]//Batch_size,
    epochs=Nepochs*2,
    callbacks = [checkpoint, early, reduceLROnPlat]
)


 Transfer Learning with: Xception: 

83689472/83683744 [==============================] - 1s 0us/step
Epoch 1/5
94/94 [==============================] - 432s 5s/step - loss: 0.5642 - accuracy: 0.7500 - val_loss: 1.6209 - val_accuracy: 0.6734

Epoch 00001: val_accuracy improved from -inf to 0.67344, saving model to Xception_weights.best.hdf5
Epoch 2/5
94/94 [==============================] - 408s 4s/step - loss: 0.5622 - accuracy: 0.7500 - val_loss: 1.2191 - val_accuracy: 0.6766

Epoch 00002: val_accuracy improved from 0.67344 to 0.67656, saving model to Xception_weights.best.hdf5
Epoch 3/5
16/94 [====>.........................] - ETA: 6:22 - loss: 0.5616 - accuracy: 0.7500

In [ ]:
def plot_trainig_history(History):
    fig, ax = plt.subplots(1, 2, figsize=(20,5))
    ax[0].set_title('loss')
    ax[0].plot(History.epoch, History.history["loss"], label="Train loss")
    ax[0].plot(History.epoch, History.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(History.epoch, History.history["accuracy"], label="Train acc")
    ax[1].plot(History.epoch, History.history["val_accuracy"], label="Validation acc")
    ax[0].legend()
    ax[1].legend();
    plt.show()

plot_trainig_history(History)

# || Testing

In [ ]:
# Load Best model weights
model.load_weights(BestModelWeightsPath)

def process_test_data():
    test_data = []
    for path in tqdm(test["img_path"].values):
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, Target_shape)
        label = test[test["img_path"] == path]["category"].values[0]
        category_cat = test[test["img_path"] == path]["category_cat"].values[0]
        test_data.append([np.array(img), label, category_cat])
    return test_data

def some_function(x):
    a = np.zeros(x.shape)
    a[np.argmax(x, axis=0)] = 1
    return a

predictions = []
true_categories = []
# Testing Model on Test Data
test_data = process_test_data()
fig =plt.figure(figsize=(20, 12))
for num, data in enumerate(test_data[:30]):
    true_label = data[1]
    true_category = data[2]
    img_data = data[0]
    
    y = fig.add_subplot(5, 6, num+1)
    data = np.expand_dims(img_data, axis=0)
    model_out = model.predict([data])[0]
    predictions.append(some_function(model_out))
    true_categories.append(true_category)
    
    most_likely_class_index = int(np.argmax(model_out))
    class_likelihood = model_out[most_likely_class_index]

    # Get the name of the most likely class
    class_label = CATEGORIES[most_likely_class_index]
    
    y.imshow(img_data)
    plt.title(class_label + "/" + true_label)
    y.axes.get_xaxis().set_visible(False);
    y.axes.get_yaxis().set_visible(False);
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
accuracy_score(np.array(predictions), np.array(true_categories))